In [6]:
import pandas as pd

ticker = 'AAPL'
# Load the options and stocks CSV files
options_df = pd.read_csv('2013-01-03options.csv')
stocks_df = pd.read_csv('2013-01-03stocks.csv')

options_df = options_df[options_df['underlying'] == ticker]
stocks_df = stocks_df[stocks_df['symbol'] == ticker]


# Rename columns in stocks_df for consistency
stocks_df = stocks_df.rename(columns={'symbol': 'underlying', 'close': 'stock_price'})

# Select relevant columns from the options file
options_df = options_df[['quote_date', 'underlying', 'expiration', 'type', 'strike', 'bid', 'ask', 'open_interest', 
                         'delta', 'gamma', 'theta', 'vega', 'implied_volatility']]

# Merge the options data with the stock prices based on the underlying symbol
merged_df = pd.merge(options_df, stocks_df[['underlying', 'stock_price']], on='underlying')

# Rename columns for consistency
merged_df = merged_df.rename(columns={
    'quote_date': 'date',
    'underlying': 'act_symbol',
    'type': 'call_put',
    'implied_volatility': 'vol'
})

# Add a rho column (since it's missing, we'll initialize it with 0)
# merged_df['rho'] = 0

# Reorder columns to match the desired output
final_df = merged_df[['date', 'act_symbol', 'expiration', 'strike', 'call_put', 'bid', 'ask', 'vol', 'delta', 'gamma', 
                      'theta', 'vega', 'stock_price']]

# Filter rows based on call_put and stock price vs. strike
final_df = final_df[
    ((final_df['call_put'] == 'call') & (final_df['stock_price'] > final_df['strike'])) |
    ((final_df['call_put'] == 'put') & (final_df['stock_price'] < final_df['strike']))
]

# Reset the index of the filtered DataFrame
final_df.reset_index(drop=True, inplace=True)

# Save the final DataFrame to a new CSV file
final_df.to_csv('merged_options_stocks.csv', index=False)

print("CSV file created successfully.")


CSV file created successfully.


In [ ]:
# import pandas as pd
# # backtest_results = pd.read_csv('', nrows=5)
# import pandas as pd

# # Specify the path to your CSV file
# file_path = '/Users/saeedbidi/Downloads/post-no-preference_options_master_option_chain.csv'

# # Create an empty list to hold the filtered data
# filtered_data = []

# # Read the CSV in chunks
# for chunk in pd.read_csv(file_path, chunksize=10000):  # Adjust chunksize as needed
#     filtered_chunk = chunk[chunk['act_symbol'] == 'AAPL']
#     filtered_data.append(filtered_chunk)

# # Concatenate the filtered chunks into a single DataFrame
# filtered_df = pd.concat(filtered_data, ignore_index=True)

# # Display the filtered DataFrame
# print(filtered_df)


In [ ]:
# # Function to obtain specific day price of stocks
# import yfinance as yf
# import pandas as pd

# def get_previous_trading_day(date):
#     """
#     Get the previous trading day.
    
#     Args:
#         date (str): The date to check in 'YYYY-MM-DD' format.
    
#     Returns:
#         str: Previous trading day in 'YYYY-MM-DD' format.
#     """
#     date = pd.to_datetime(date)
#     while True:
#         date -= pd.DateOffset(days=1)
#         if date.weekday() < 5:  # 0=Monday, 1=Tuesday, ..., 4=Friday
#             return date.strftime('%Y-%m-%d')

# def get_stock_price(ticker, date):
#     """
#     Fetch stock price for a specific date.
    
#     Args:
#         ticker (str): Stock ticker symbol.
#         date (str): The date to fetch stock price for in 'YYYY-MM-DD' format.
    
#     Returns:
#         float: The stock price for the given date.
#     """
#     try:
#         # Fetch data for the specified date
#         stock_data = yf.download(ticker, start=date, end=pd.to_datetime(date) + pd.DateOffset(days=1))
        
#         if stock_data.empty:
#             # If no data is found, get the previous trading day
#             previous_date = get_previous_trading_day(date)
#             print(f"No data found for {date}. Checking previous trading day: {previous_date}")
#             stock_data = yf.download(ticker, start=previous_date, end=pd.to_datetime(previous_date) + pd.DateOffset(days=1))
        
#         if stock_data.empty:
#             raise ValueError("No stock data found for the specified date or the previous trading day.")
        
#         if 'Adj Close' in stock_data.columns:
#             return stock_data['Adj Close'].iloc[0]  # Get the adjusted close price
#         else:
#             raise ValueError("Adjusted Close data is not available.")

#     except Exception as e:
#         print(f"Error fetching stock data: {e}")
#         return None

# # Example usage
# ticker = 'AAPL'
# specific_date = '2024-09-25'
# stock_price_on_date = get_stock_price(ticker, specific_date)
# print(f"Stock price of {ticker} on {specific_date}: {stock_price_on_date}")
